In [ ]:
print('Setup complete.')

# Lab 01: Advanced Prompting Techniques

## Learning Objectives
- Understand and implement Chain-of-Thought (CoT) prompting
- Apply Self-Consistency to improve the reliability of CoT
- Explore the conceptual framework of Tree of Thoughts (ToT)
- Learn how to structure prompts to elicit more complex reasoning from LLMs

## Setup

In [ ]:
import re
from typing import List, Dict, Any
from collections import Counter
import random

## Part 1: Chain-of-Thought (CoT) Prompting

Chain-of-Thought prompting encourages the LLM to break down a problem into intermediate steps, mimicking a human-like reasoning process. This is particularly effective for arithmetic, commonsense, and symbolic reasoning tasks.

In [ ]:
class MockLLM:
    """A mock LLM to simulate responses based on prompt structure."""
    def generate(self, prompt: str) -> str:
        # If the prompt asks for steps, the LLM is more likely to produce them.
        if 'step by step' in prompt.lower():
            return (
                'Step 1: Identify the number of apples. There are 5 apples.
'
                'Step 2: Identify the number of apples given away. 2 apples were given away.
'
                'Step 3: Subtract the given apples from the initial amount. 5 - 2 = 3.
'
                'The answer is 3.'
            )
        else:
            # Without CoT, the model might guess incorrectly.
            return 'The answer is 4.'

llm = MockLLM()

# Standard Prompt
problem = 'Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?'
standard_prompt = f'{problem}\nA:'

# Chain-of-Thought Prompt
cot_prompt = f'{problem}\nA: Let\'s think step by step.'

print("--- Standard Prompt Response ---")
print(llm.generate(standard_prompt))

print("
--- Chain-of-Thought Prompt Response ---")
print(llm.generate(cot_prompt))

## Part 2: Self-Consistency

Self-Consistency improves on CoT by generating multiple reasoning paths and then taking a majority vote on the final answer. This makes the model more robust to individual reasoning errors.

In [ ]:
class MockLLMWithRandomness(MockLLM):
    """A mock LLM that can produce slightly different reasoning paths."""
    def generate(self, prompt: str) -> str:
        if 'step by step' in prompt.lower():
            # Simulate different reasoning paths with some randomness
            path_1 = 'Step 1: 5 initial balls. Step 2: 2 cans * 3 balls/can = 6 balls. Step 3: 5 + 6 = 11. The answer is 11.'
            path_2 = 'Step 1: 2 cans * 3 balls = 6. Step 2: 6 + 5 = 11. The answer is 11.'
            path_3 = 'Step 1: Roger started with 5. Step 2: He added 2 cans, so 5+2=7. The answer is 7.' # A reasoning error
            return random.choice([path_1, path_1, path_2, path_3]) # Skew towards the correct answer
        return 'The answer is 8.'

def self_consistency_decode(prompt: str, llm: MockLLMWithRandomness, num_paths: int) -> str:
    responses = [llm.generate(prompt) for _ in range(num_paths)]
    
    # Extract the final answer from each reasoning path
    answers = []
    for res in responses:
        match = re.search(r'The answer is (\d+)', res)
        if match:
            answers.append(match.group(1))
            
    if not answers:
        return 'Could not determine an answer.'
        
    # Take the majority vote
    final_answer = Counter(answers).most_common(1)[0][0]
    return final_answer

sc_llm = MockLLMWithRandomness()
final_answer = self_consistency_decode(cot_prompt, sc_llm, num_paths=5)

print("--- Self-Consistency Result ---")
print(f'The final answer after majority vote is: {final_answer}')

## Part 3: Tree of Thoughts (ToT) - Conceptual

Tree of Thoughts extends CoT by exploring multiple reasoning paths in a tree-like structure. At each step, the model generates multiple possible "thoughts" and uses a deliberate search algorithm (like breadth-first or depth-first search) to explore the most promising paths.

In [ ]:
# ToT is an algorithm that uses an LLM, not just a prompt. We'll represent it conceptually.

@dataclass
class ThoughtNode:
    state: str  # The current state of the problem (e.g., 'Roger has 5 balls')
    children: List['ThoughtNode'] = field(default_factory=list)
    evaluation: float = 0.0 # How promising this path is

def tot_search(problem: str, llm: MockLLM, max_depth: int, breadth: int) -> str:
    """A conceptual implementation of a Tree of Thoughts search."""
    root = ThoughtNode(state=problem)
    frontier = [root]
    
    for depth in range(max_depth):
        next_frontier = []
        for node in frontier:
            # 1. Generate multiple next steps (thoughts)
            for _ in range(breadth):
                # In a real system, the prompt would ask for the next logical step
                next_step_prompt = f'Current state: {node.state}. What are возможные next steps?'
                # Mocking the generation of next steps
                next_thought = random.choice([
                    'Calculate balls in cans (2*3=6)', 
                    'Add cans to balls (5+2=7)', 
                    'Sum everything (5+2+3=10)'
                ])
                child = ThoughtNode(state=f'{node.state}, then {next_thought}')
                
                # 2. Evaluate the new state (heuristic)
                if '11' in child.state or '6' in child.state: # Good heuristic
                    child.evaluation = 0.8
                elif '7' in child.state: # Bad heuristic
                    child.evaluation = 0.2
                else:
                    child.evaluation = 0.5
                
                node.children.append(child)
                next_frontier.append(child)
        
        # 3. Prune the tree, keeping only the most promising nodes
        frontier = sorted(next_frontier, key=lambda n: n.evaluation, reverse=True)[:breadth]

    # Return the state of the best leaf node
    best_path = sorted(frontier, key=lambda n: n.evaluation, reverse=True)[0]
    return best_path.state

print("--- Tree of Thoughts (Conceptual Search) ---")
best_reasoning_path = tot_search(problem, llm, max_depth=2, breadth=2)
print(f'Most promising reasoning path found:
{best_reasoning_path}')

## Exercises

1. **Create a Zero-Shot CoT Prompt**: Modify the CoT prompt to be "zero-shot" by simply appending "Let's think step by step" to the question without providing any examples. How does the mock LLM handle this? (Our current one does, but in reality, performance might vary).
2. **Adjust Self-Consistency Parameters**: Change the `num_paths` in the `self_consistency_decode` function. What happens if you use a small number (e.g., 2) versus a large number (e.g., 20)?
3. **Improve the ToT Evaluator**: The `evaluation` heuristic in our `tot_search` is very simple. Propose a more sophisticated way to evaluate a thought. For example, you could use the LLM itself to score how logical a step is.

## Summary

You learned:
- How **Chain-of-Thought (CoT)** prompting guides an LLM to produce a reasoning process, improving accuracy.
- How **Self-Consistency** builds on CoT by sampling multiple reasoning paths and using a majority vote to find a more reliable answer.
- The conceptual framework of **Tree of Thoughts (ToT)**, a deliberate search algorithm that explores a tree of reasoning steps to solve more complex problems.